# Noise-contrastive estimation: A new estimation principle for unnormalized statistical models

Guntmann imply that say we want to model some complex solution using statistical models. For high dimensionality problems it is close to impossible to calculate them. And to calculate them we need to compute the integral of the Probability Density Function (PDF). Even if we manage to find the the PDF it is hard to impossible to find its integral due to complexity.

One of examples is using Energy Based models or multi-layer networks which reference Neural Networks. Neural networks can be considered as one of the examples of PDF that can solve and model very complex tasks. It is both impossible to compute the integral of the Network because of its high complexity and computationally very expensive as well, say by using Softmax, given large number of classes present in the dataset as calculating softmax can be considered a form of finding Z, specifically for discrete and finite scenarios. 

The softmax function computes the normalization factor directly in its denominator (which sums up the exponentials of all outputs) to ensure the outputs form a valid probability distribution. This sum effectively acts as Z by normalizing the values so that their total is 1. So, in natual language processing or image detection with a lot of classes it quickly becomes infeasible / ineffective to compute directly